In [1]:
import json
import ast
import re
import csv

# 1. Chuyển từ dữ liệu text thành file csv

In [2]:
def extract_json_blocks(text: str):
    blocks = []

    pattern = r"```(?:json|python)?\s*(.*?)```"
    matches = re.findall(pattern, text, flags=re.DOTALL)

    for m in matches:
        candidate = m.strip()
        if ("[" in candidate and "]" in candidate) or ("{" in candidate and "}" in candidate):
            blocks.append(candidate)

    if not blocks:
        cleaned = text.strip()
        if cleaned:
            blocks = [cleaned]

    return blocks



In [3]:
def parse_block_to_python(block: str):
    block = block.strip()

    try:
        return json.loads(block)
    except Exception:
        pass

    try:
        return ast.literal_eval(block)
    except Exception:
        pass

    try:
        wrapped = "[" + block + "]"
        return json.loads(wrapped)
    except Exception:
        pass

    print(block)

    raise ValueError("Không parse được block:\n" + block[:200])

In [4]:
def normalize_record(rec: dict):
    def to_list(v):
        if v is None:
            return []
        if isinstance(v, list):
            return v
        if isinstance(v, dict):
            return [v]
        return [v]

    mapping = {
        "Tên cây": "ten_cay",
        "tên cây": "ten_cay",
        "Loại bệnh": "loai_benh",
        "loại bệnh": "loai_benh",
        "Nguyên nhân": "nguyen_nhan",
        "nguyên nhân": "nguyen_nhan",
        "Triệu chứng": "trieu_chung",
        "triệu chứng": "trieu_chung",
        "Cách điều trị": "cach_dieu_tri",
        "cách điều trị": "cach_dieu_tri",
    }

    out = {
        "ten_cay": "",
        "loai_benh": "",
        "nguyen_nhan": [],
        "trieu_chung": [],
        "cach_dieu_tri": []
    }

    for k, v in rec.items():
        if k in mapping:
            mk = mapping[k]
            if mk in ["ten_cay", "loai_benh"]:
                out[mk] = v.strip() if isinstance(v, str) else str(v)
            else:
                out[mk] = to_list(v)
        else:
            pass

    return out

In [5]:
def pipeline(txt_path: str, json_path: str, csv_path: str):
    with open(txt_path, "r", encoding="utf-8") as f:
        raw = f.read()

    blocks = extract_json_blocks(raw)

    results = []

    for block in blocks:
        if block.startswith("Dưới đây") or block.startswith("Dựa trên nội dung") or block.startswith("Từ nội dung") or block.startswith("Dựa vào nội dung"):
            print(block)
            continue
        print("---------------")
        parsed = parse_block_to_python(block)
        if isinstance(parsed, list):
            for item in parsed:
                if isinstance(item, dict):
                    results.append(normalize_record(item))
        elif isinstance(parsed, dict):
            results.append(normalize_record(parsed))
    with open(csv_path, "a", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["ten_cay", "loai_benh", "nguyen_nhan", "trieu_chung", "cach_dieu_tri"]
        )
        writer.writeheader()
        for r in results:
            writer.writerow({
                "ten_cay": r["ten_cay"],
                "loai_benh": r["loai_benh"],
                "nguyen_nhan": " | ".join(map(str, r["nguyen_nhan"])),
                "trieu_chung": " | ".join(map(str, r["trieu_chung"])),
                "cach_dieu_tri": " | ".join(map(str, r["cach_dieu_tri"])),
            })

    print(f"✔ Hoàn thành! Tổng số record: {len(results)}")
    print(f"→ JSON: {json_path}")
    print(f"→ CSV:  {csv_path}")

In [11]:
pipeline(
        txt_path="raw_data/raw_data_general_2.txt",
        json_path="clean_data_test1.json",
        csv_path="data/general_data_raw_3.csv"
    )


---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
--------

# 2. Tiền xử lý dữ liệu sau khi đã được chuyển đổi

In [12]:
csv_path_general = "data/general_data_raw_3.csv"
csv_file_test = "data/test_data.csv"

with open(csv_path_general, mode='r', encoding='utf-8', newline='') as infile, \
             open(csv_file_test, mode='w', encoding='utf-8', newline='') as outfile:

    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    # Đọc và ghi tối đa n dòng
    for i, row in enumerate(reader):
        if i >= 20:
            break
        writer.writerow(row)

In [16]:
import pandas as pd

def clean_ten_cay(name: str) -> str:
    """
    Xóa từ 'cây' hoặc 'cây ' ở đầu chuỗi.
    Không ảnh hưởng các vị trí khác.
    Ví dụ:
        'cây hoa cúc' -> 'hoa cúc'
        'cây cam' -> 'cam'
        'hoa hồng' -> 'hoa hồng' (giữ nguyên)
    """
    if not isinstance(name, str):
        return name

    name = name.strip().lower()

    # Nếu bắt đầu bằng 'cây '
    if name.startswith("cây "):
        return name[4:].strip()  # bỏ 'cây ' + strip dư thừa

    # Nếu chỉ là đúng chữ 'cây'
    if name == "cây":
        return ""

    return name



df = pd.read_csv("data/general_data_raw_3.csv")

df["ten_cay"] = df["ten_cay"].apply(clean_ten_cay)

df.to_csv("data/data_clean_1.csv", index=False)

In [21]:
# Tìm kiếm các row có tên cây và loại bệnh trùng nhau
def find_duplicates(csv_path):
    df = pd.read_csv(csv_path)

    # Tìm duplicated theo 2 cột
    dup_mask = df.duplicated(subset=["ten_cay", "loai_benh"], keep=False)

    duplicates_df = df[dup_mask].sort_values(by=["ten_cay", "loai_benh"])

    return duplicates_df

# xóa những bản ghi trùng nhau, chỉ giữ lại bản ghi đầu tiên
def remove_duplicates_keep_first(csv_path, output_path):
    df = pd.read_csv(csv_path)

    df_cleaned = df.drop_duplicates(
        subset=["ten_cay", "loai_benh"],
        keep="first"   # giữ dòng đầu tiên
    )

    df_cleaned.to_csv(output_path, index=False)
    return df_cleaned

In [22]:
dups = find_duplicates("data/data_clean_1.csv")
dups  # xem các hàng trùng nhau

,ten_cay,loai_benh,nguyen_nhan,trieu_chung,cach_dieu_tri
67,cao su,Bệnh nấm hồng,Nấm Corticium salmonicolor. Phát triển mạnh tr...,Giai đoạn đầu trên bề mặt vỏ cây xuất hiện lớp...,"Phòng ngừa: Chọn giống kháng bệnh, canh tác đú..."
99,cao su,Bệnh nấm hồng,Tác nhân chính: Nấm Corticium salmonicolor. | ...,{'Trên thân và cành cây': ['Ban đầu: Các sợi n...,{'Biện pháp canh tác phòng trừ': ['Xây dựng hệ...
63,cao su,Bệnh phấn trắng,Do nấm Oidium heveae Steinm hoặc các loại nấm ...,Giai đoạn đầu (lá non màu đồng tím): Lá cây nh...,"{'phòng ngừa và canh tác': ['Tỉa cành, vệ sinh..."
66,cao su,Bệnh phấn trắng,Nấm Oidium Hevea. Phát triển mạnh trong thời t...,"Lá non có màu đồng tím, nhăn nheo và có lớp bộ...","Phòng ngừa: Chọn giống kháng bệnh, canh tác đú..."
637,chanh,Đốm lá,Nấm gây hại (thuốc Phy Fusaco có công dụng điề...,Không được mô tả chi tiết trong bài viết.,"{'Tên thuốc': 'Phy Fusaco', 'Hướng dẫn sử dụng..."
...,...,...,...,...,...
539,đậu cove,Đốm lá,"Không được đề cập chi tiết trong bài viết, như...",Không được đề cập chi tiết trong bài viết.,Phòng ngừa và điều trị bằng thuốc sinh học Ant...
548,đậu cove,Đốm lá,Do nấm Cercospora cruenta xâm nhập và gây hại....,"Trên lá: Xuất hiện các vết đốm vòng, trung tâm...",{'phương pháp chung': ['Thu gom tàn dư thực vậ...
135,ớt,Bệnh thán thư,Do nấm Collectotrichum (đặc biệt là Collectotr...,{'Tổng quát': 'Có thể xuất hiện trên bất kỳ bộ...,{'Phòng ngừa': ['Chọn thời vụ trồng hợp lý (ít...
948,ớt,Bệnh thán thư,Không đề cập chi tiết trong bài viết (chỉ liệt...,Không đề cập chi tiết trong bài viết.,Không đề cập chi tiết trong bài viết.


In [23]:
remove_duplicates_keep_first("data/data_clean_1.csv", "data/data_clean_2.csv")

,ten_cay,loai_benh,nguyen_nhan,trieu_chung,cach_dieu_tri
0,mai vàng,Bệnh cháy lá (hay còn gọi là cháy bìa lá),"Do các loại nấm bệnh tấn công, đặc biệt là nấm...","Nấm bệnh thường phát sinh vào cuối mùa thu, tr...",{'Phương pháp canh tác phòng trừ': ['Lựa chọn ...
1,sầu riêng,Bệnh phấn trắng,Do nấm Oidium sp. gây ra. | Nấm phát triển thu...,**Qua lá:** Xuất hiện lớp bụi màu trắng mịn và...,**Biện pháp phòng ngừa:** | - Chọn giống sầu r...
2,sầu riêng,Khô cành (còn gọi là khô đọt),Tác nhân chính: Nấm khuẩn Rhizoctonia solani. ...,"Bệnh xuất hiện từ cành nhỏ, cành phía dưới tán...",{'Loại biện pháp': 'Kỹ thuật canh tác phòng bệ...
3,khoai mì (sắn),Thối củ (còn gọi là lở cổ rễ),{'Trực tiếp': 'Do nấm Phytopythium helicoides ...,Xuất hiện trên vị trí cổ rễ (phần thân sát gốc...,{'Khi phát hiện bệnh (xử lý ngay)': ['Nhổ bỏ t...
4,dừa,Cháy lá,Do nấm bệnh Pestalozzia palmarum và Helminthos...,Khi nhiễm nấm Pestalozzia palmarum: Trên lá xu...,**Canh tác phòng ngừa:** | - Bổ sung dinh dưỡn...
...,...,...,...,...,...
1258,hoa lan,Đốm nâu,Do nấm Cercospora sp. | Các loài côn trùng như...,Xuất hiện các vết bệnh ở cả hai mặt của lá. | ...,{'Phương pháp canh tác phòng ngừa': ['Thường x...
1259,mít,Bệnh thán thư,Tác nhân trực tiếp: Nấm Colletotrichum gloeosp...,Trên lá: Xuất hiện đốm nâu nhỏ kèm hiện tượng ...,{'Kỹ thuật canh tác (Phòng bệnh)': ['Tạo độ th...
1260,mít,Xoăn lá,"Tác nhân tự nhiên: | - Sâu bệnh, sâu ăn lá hại...","Lá phủ màu vàng. | Lá xoăn, bị rụng. | Hình dạ...",{'Phòng bệnh': ['Cắt bỏ những cây bị bệnh trướ...
1261,ớt,Bệnh héo xanh (Pseudomonas solanacearum),Vi khuẩn Pseudomonas solanacearum là tác nhân ...,Thường xảy ra ở thời kỳ sinh trưởng và kết trá...,{'Biện pháp canh tác phòng bệnh': ['Áp dụng ph...
